In [ ]:
"""

import time
start = time.time()
"""


# 고려 사항
##### 모델 사이즈 : 원본 사이즈는 112 X 112
##### 표본의 크기 : Max size ( 타켓별 랜덤샘플링 할때의 최대 표본 크기)
##### 주성분 크기 : pca
##### svc 파라미터 : gamma, c 값 (그리드서치이용)

In [1]:
# 샘플링
model_size = (56,56)
sample_size = 5

#모델 pca,svc
pca_size = 100
svc_C = 5
svc_gamma = 0.01

#모델 bagging
estimator_size = 6
sample_ratio = 5000

라이브러리 불러오기

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline
import joblib
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from numpy.linalg import norm
from numpy import dot
from sklearn.ensemble import BaggingClassifier

데이터 불러오기

In [3]:

def sampling_func(data, max_size):
    N = len(data)
    sample_n = len(data) # integer
    if sample_n > max_size:
        sample = data.take(np.random.permutation(N)[:max_size])
    else:
        sample = data.take(np.random.permutation(N)[:len(data)])
    return sample

ID_List = pd.read_table('/Users/heogeonhyeock/Downloads/inha_data/inha_data/ID_List.txt',
                       sep = " ",
                       names = ["name", "path"])

ID_List_sample = ID_List.groupby('name', group_keys = False
                                ).apply(sampling_func, max_size = sample_size).reset_index()


In [4]:

train_img = []
train_name = []

for index in range(len(ID_List_sample)):
    A = "/Users/heogeonhyeock/Downloads/inha_data/inha_data/" + ID_List_sample['path'][index]
    file = cv2.resize(cv2.imread(A,0), model_size).reshape(1,-1)[0]
    train_img.append(file) # 리스트에 이미지 추가
    train_name.append(ID_List_sample['name'][index])

train_img = np.array(train_img)
train_name = np.array(train_name)


train = dict(data = train_img,
            target = train_name)


In [5]:
"""
# 데이터 불러오기 csv
train_data_max5 = pd.read_csv("/Users/heogeonhyeock/Desktop/train_data_max5.csv", index_col=False) # 데이터 불러오기
train_data = dict(data = np.array(train_data_max5.drop(['Unnamed: 0'], axis = 'columns'), dtype='uint8'),
                  name = np.array(train_data_max5['Unnamed: 0']))
"""

'\n# 데이터 불러오기 csv\ntrain_data_max5 = pd.read_csv("/Users/heogeonhyeock/Desktop/train_data_max5.csv", index_col=False) # 데이터 불러오기\ntrain_data = dict(data = np.array(train_data_max5.drop([\'Unnamed: 0\'], axis = \'columns\'), dtype=\'uint8\'),\n                  name = np.array(train_data_max5[\'Unnamed: 0\']))\n'

모델

In [6]:
# 모델 생성
pca = RandomizedPCA(n_components = pca_size, whiten=True)
svc = SVC(kernel = "rbf", class_weight = "balanced",probability=True, 
          C = svc_C, gamma= svc_gamma,cache_size=10000)
model = BaggingClassifier(base_estimator = make_pipeline(pca,svc),
                          n_estimators = estimator_size,
                          max_samples = sample_ratio,
                          verbose=1,
                          n_jobs=6)
                          
                          
                          
                          


In [7]:
"""
# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

param_grid = {'svc__C' : [1,5,10,15],
              'svc__gamma' : [0.001,0.005,0.01,0.05]}

grid = GridSearchCV(model, param_grid, cv=3 ,n_jobs=6)
grid.fit(train['data'], train['target'])

model = grid.best_estimator_
"""

# 모델 훈련
model.fit(train['data'], train['target'])

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed:  6.9min remaining: 13.9min
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:  7.0min finished


BaggingClassifier(base_estimator=Pipeline(steps=[('pca',
                                                  PCA(n_components=100,
                                                      whiten=True)),
                                                 ('svc',
                                                  SVC(C=5, cache_size=10000,
                                                      class_weight='balanced',
                                                      gamma=0.01,
                                                      probability=True))]),
                  max_samples=5000, n_estimators=6, n_jobs=6, verbose=1)

In [8]:
# 모델 저장
joblib.dump(model, './Inha_svc_Bagging_8.12.pkl')

['./Inha_svc_Bagging_8.12.pkl']

모델 적용

In [ ]:
"""
# 테스트 데이터
left_test_img = []
right_test_img = []

for index in range(6000):
    A = "/Users/heogeonhyeock/Downloads/inha_data/inha_data/test/left_face_" + str(index) + ".jpg"
    B = "/Users/heogeonhyeock/Downloads/inha_data/inha_data/test/right_face_" + str(index) + ".jpg" 
    
    file_left = cv2.resize(cv2.imread(A,0), model_size).reshape(1,-1)[0]
    
    file_right = cv2.resize(cv2.imread(B,0), model_size).reshape(1,-1)[0]

    left_test_img.append(file_left)
    right_test_img.append(file_right)
"""

In [14]:
# 데이터 불러오기
left_test_img = np.array(pd.read_csv("/Users/heogeonhyeock/Desktop/left_test_data"))
right_test_img = np.array(pd.read_csv("/Users/heogeonhyeock/Desktop/right_test_data"))

In [15]:
# 정답 저장 코사인 유사도

predict = []

predict_left_prob = model.predict_proba(left_test_img)
predict_right_prob = model.predict_proba(right_test_img)

def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))
    
for i in range(len(predict_left_prob)):
    predict_cos = cos_sim(predict_left_prob[i],predict_right_prob[i])
    predict.append(predict_cos)

submission = pd.read_csv("/Users/heogeonhyeock/Downloads/inha_data/inha_data/sample_submission.csv")
submission["answer"] = predict[:]

path = '/Users/heogeonhyeock/Downloads/inha_data/inha_data'
os.chdir(path)
submission.to_csv("submission.csv", index = False)

submission

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   6 | elapsed: 38.3min remaining: 76.6min


KeyboardInterrupt: 

In [ ]:
"""
# 정답 저장2 auc
#predict_left = model.predict(left_test_img)
#predict_right = model.predict(right_test_img)

predict2 = []

#predict_left_prob = model.predict_proba(left_test_img)
predict_right = model.predict(right_test_img)
predict_right_onhot = to_categorical(predict_right)

for i in range(len(predict_left_prob)):
    A = roc_auc_score(predict_right_onehot[i], predict_left_prob[i], average='macro', 
              sample_weight=None, max_fpr=None, multi_class='ovr', labels=None)
    predict2.append(A)

submission = pd.read_csv("/Users/heogeonhyeock/Downloads/inha_data/inha_data/sample_submission.csv")
submission["answer"] = predict2[:]

path = '/Users/heogeonhyeock/Downloads/inha_data/inha_data'
os.chdir(path)
submission.to_csv("submission_auc.csv", index = False)

submission
"""